# Predicción del estrés financiero

# Librerías

In [58]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, validation_curve
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor


%matplotlib inline

# Carga del dataset

In [59]:
#pip install openpyxl

In [60]:
dataset = pd.read_excel('./data/product/product.xlsx', engine='openpyxl')

dataset

,TD_div_TA,WC_div_TA,EBITDA_div_TA,lnTA,isFamiliar,z_cat
0,0.563004,0.039019,0.050417,19.051693,-1,1
1,0.565019,0.031774,0.019354,19.030923,-1,1
2,0.574824,0.041247,0.059132,19.070886,-1,1
3,0.619957,0.231849,0.043954,19.686780,0,2
4,0.625639,0.191468,0.030919,19.710178,0,2
...,...,...,...,...,...,...
4084,0.914869,0.801310,-0.337460,10.682434,-1,2
4085,1.254419,0.560703,-0.392152,10.306232,-1,2
4086,0.516316,0.543594,-0.062332,14.161339,-1,2
4087,0.620462,0.399910,-0.205751,14.039860,-1,2


In [61]:
X = pd.DataFrame(dataset[dataset.columns[:-1]])
Y = dataset[dataset.columns[-1]]

In [62]:
X.head()

,TD_div_TA,WC_div_TA,EBITDA_div_TA,lnTA,isFamiliar
0,0.563004,0.039019,0.050417,19.051693,-1
1,0.565019,0.031774,0.019354,19.030923,-1
2,0.574824,0.041247,0.059132,19.070886,-1
3,0.619957,0.231849,0.043954,19.686780,0
4,0.625639,0.191468,0.030919,19.710178,0


In [63]:
Y.head()

0    1
1    1
2    1
3    2
4    2
Name: z_cat, dtype: int64

# Semilla

In [64]:
myseed=1234

# Métricas

In [65]:
from sklearn.metrics import make_scorer
from sklearn.metrics import average_precision_score, recall_score

recall_scorer = make_scorer(recall_score, pos_label=1,labels=[0,1])
spec_scorer = make_scorer(recall_score, pos_label=0,labels=[0,1])
pr_auc_scorer = make_scorer(average_precision_score)

accuracy = []
f1 = []
precision = []
recall = []
spec=[]
roc_auc = []
prec_rec = []

dicci = {"accuracy":"accuracy", "average_precision":pr_auc_scorer,
         "f1":"f1", "precision":"precision", "recall":recall_scorer,
         "spec":spec_scorer,"roc_auc":"roc_auc"}

def score(scores):
    accuracy.append(str(np.around(np.mean(scores.get("test_accuracy")),3)) + " ("
                + str(np.around(np.std(scores.get("test_accuracy")), 3))+ ")")
    f1.append(str(np.around(np.mean(scores.get("test_f1")),3)) + " ("
                + str(np.around(np.std(scores.get("test_f1")), 3))+ ")")
    precision.append(str(np.around(np.mean(scores.get("test_precision")),3)) + " ("
                + str(np.around(np.std(scores.get("test_precision")), 3))+ ")")
    recall.append(str(np.around(np.mean(scores.get("test_recall")),3)) + " ("
                + str(np.around(np.std(scores.get("test_recall")), 3))+ ")")
    spec.append(str(np.around(np.mean(scores.get("test_spec")),3)) + " ("
                + str(np.around(np.std(scores.get("test_spec")), 3))+ ")")
    roc_auc.append(str(np.around(np.mean(scores.get("test_roc_auc")),3)) + " ("
                + str(np.around(np.std(scores.get("test_roc_auc")), 3))+ ")")

# Regresión logística

Los algoritmos ya tienen integrados la validación cruzada estratificada, una vez limpien los datos y elaboren las variables. Prueben con este para empezar

In [66]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

#solvers = ['liblinear','lbfgs','saga','sag']
#penal = ['l1','l2','elasticnet']
logistic = LogisticRegressionCV(random_state=myseed, max_iter=200, solver='liblinear',penalty='l1')
scaler = StandardScaler()
scores = cross_validate(logistic, X,Y, cv=10, scoring=dicci)
score(scores)
scores

c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_ranking.py", line 261, in

{'fit_time': array([3.67843771, 2.66254807, 2.70638728, 2.7761066 , 2.77898169,
        2.75600982, 2.83092666, 2.73308253, 2.7203753 , 3.19578743]),
 'score_time': array([0.01000023, 0.0090003 , 0.01000214, 0.00899768, 0.00999045,
        0.00899434, 0.00900149, 0.00998402, 0.00999093, 0.01021123]),
 'test_accuracy': array([0.76528117, 0.80195599, 0.81418093, 0.81662592, 0.85085575,
        0.78973105, 0.82151589, 0.80195599, 0.82396088, 0.85294118]),
 'test_average_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_recall': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_spec': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_roc_auc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}

In [67]:
from sklearn.model_selection import cross_val_predict
logistic = LogisticRegressionCV(random_state=myseed, max_iter=200, solver='liblinear',penalty='l1')
scaler = StandardScaler()
prediccioneslogistic = cross_val_predict(logistic, X,Y, cv=10)
prediccioneslogisticproba = cross_val_predict(logistic, X,Y, cv=10, method="predict_proba")

# Random Forest

La idea básica del bagging es utilizar el conjunto de entrenamiento original para generar centenares o miles de conjuntos similares usando muestreo con reemplazo. En este concepto está basado el algoritmo Random Forest, la combinación de varios árboles de decisión, cada uno entrenado con una realización diferente de los datos. La decisión final del clasificador combinado (el Random Forest) se toma por mayoría, dando el mismo peso a todas las decisiones parciales tomadas por los clasificadores base (los árboles).

In [68]:
#depth = [10,25,50,75,100,125]
#estimadores= [1000]

rfc = RandomForestClassifier(random_state=myseed, max_depth=32, n_estimators=954)
scoresrfc = cross_validate(rfc, X, Y, cv=10, scoring=dicci)
score(scoresrfc)
scoresrfc

c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_ranking.py", line 261, in

{'fit_time': array([5.97980261, 5.96098924, 6.00977135, 6.04617143, 6.08586216,
        5.92904115, 5.89960575, 5.90795469, 5.93912482, 5.9699986 ]),
 'score_time': array([0.17999458, 0.18899965, 0.28600001, 0.19403243, 0.19220281,
        0.19095588, 0.18799162, 0.1880362 , 0.18500161, 0.18499899]),
 'test_accuracy': array([0.78239609, 0.82640587, 0.83863081, 0.86797066, 0.85574572,
        0.78239609, 0.83374083, 0.82885086, 0.85574572, 0.89460784]),
 'test_average_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_f1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_recall': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_spec': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_roc_auc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}

In [69]:
rfc = RandomForestClassifier(random_state=myseed, max_depth=32, n_estimators=954)
prediccionesrfc = cross_val_predict(rfc, X, Y, cv=10)
prediccionesrfcproba = cross_val_predict(rfc, X,Y, cv=10, method="predict_proba")

# KNN

In [70]:
#algoritmo = ['auto','ball_tree','kd_tree','brute']
#p = [1,2]
#metric = ['euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis']

neigh = KNeighborsClassifier(n_neighbors=2, algorithm='auto',p=1, weights="distance")
scoresneigh = cross_validate(neigh, X, Y, cv=10, scoring=dicci)
score(scoresneigh)
scoresneigh
print(str(np.around(np.mean(scoresneigh['test_roc_auc']),3)))

nan


c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_ranking.py", line 261, in

In [71]:
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='auto',p=1, weights="distance")
prediccionesneigh = cross_val_predict(neigh, X, Y, cv=10)
prediccionesneighproba = cross_val_predict(neigh, X,Y, cv=10, method="predict_proba")

# SVM

In [72]:
from sklearn.model_selection import cross_validate
#kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
#C = [1.0,5.0,10.0,50.0,100.0]
#gammas = [0.1,0.01,0.05,0.001,0.005]

svm = SVC(gamma=0.0023420730885665587,probability=True, C=55.56346718268796)
scoressvm = cross_validate(svm, X, Y, cv=10, scoring=dicci)
score(scoressvm)
print(str(np.around(np.mean(scoressvm['test_f1']),3)))
print(str(np.around(np.mean(scoressvm['test_spec']),3)))
print(str(np.around(np.mean(scoressvm['test_roc_auc']),3)))

c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_ranking.py", line 261, in

nan
nan
nan


c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\model_selection\_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nelso\miniconda3\envs\vc_project\Lib\site-packages\sklearn\metrics\_ranking.py", line 261, in

In [73]:
svm = SVC(gamma=0.0023420730885665587,probability=True, C=55.56346718268796)
prediccionessvm = cross_val_predict(svm, X, Y, cv=10)
prediccionessvmproba = cross_val_predict(svm, X, Y, cv=10, method="predict_proba")

# Resultados

In [74]:
conjunto1 = pd.DataFrame(columns=["accuracy", "f1_score", "precision","recall","specificity","roc_auc"])
conjunto1["accuracy"] = accuracy
conjunto1["f1_score"] = f1
conjunto1["precision"] = precision
conjunto1["recall"] = recall
conjunto1["specificity"] = spec
conjunto1["roc_auc"] = roc_auc
conjunto1.index = ["LOGIT","RFC", "KNN","SVM"]
conjunto1.to_csv("conjunto1.csv")

In [75]:
conjunto1

,accuracy,f1_score,precision,recall,specificity,roc_auc
LOGIT,0.814 (0.025),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan)
RFC,0.837 (0.033),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan)
KNN,0.666 (0.049),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan)
SVM,0.813 (0.029),nan (nan),nan (nan),nan (nan),nan (nan),nan (nan)


In [76]:
conprobas = []

conprobas.append(prediccioneslogisticproba)
conprobas.append(prediccionesneighproba)
conprobas.append(prediccionesrfcproba)
conprobas.append(prediccionessvmproba)

In [77]:
clasificadores = ["LOGIT","RFC", "KNN","SVM"]
colores = ["brown","coral","gold","green"]

In [78]:
#roc_auc[0] = '0.805 (0.018)'
#roc_auc[1] = '0.705 (0.038)'
#roc_auc[2] = '0.901 (0.023)'
#roc_auc[3] = '0.890 (0.018)'
#roc_auc[4] = '0.976 (0.008)'
#roc_auc[5] = '0.972 (0.015)'

In [82]:
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize


def plot_roc3(y_true, probis,testnn, colores,nombres,title='ROC Curves',
                   plot_micro=True, plot_macro=True, classes_to_plot=None,
                   ax=None, figsize=None, cmap='nipy_spectral',
                   title_fontsize="large", text_fontsize="medium"):

    testnn = np.array(testnn)
    y_true = np.array(y_true)
    classes = np.unique(y_true)

    for j in range(6):
        probis[j] = np.array(probis[j])
        probas = probis[j]
        if classes_to_plot is None:
            classes_to_plot = classes
        if ax is None:
            fig, ax = plt.subplots(1, 1, figsize=figsize)
            ax.set_title(title, fontsize=title_fontsize)
        fpr_dict = dict()
        tpr_dict = dict()
        indices_to_plot = np.in1d(classes, classes_to_plot)

        if j ==0:
            for i, to_plot in enumerate(indices_to_plot):
                fpr_dict[i], tpr_dict[i], _ = roc_curve(testnn, probas[:, i],
                                                pos_label=classes[i])

            if plot_macro:
                ax.plot(fpr_dict[0], tpr_dict[0], label='ROC Curve - ' + nombres[j] + ' - ' + roc_auc[j], color=colores[j], linestyle=':', linewidth=4)
        else:
            for i, to_plot in enumerate(indices_to_plot):
                fpr_dict[i], tpr_dict[i], _ = roc_curve(y_true, probas[:, i],
                                                pos_label=classes[i])
            if plot_macro:
                ax.plot(fpr_dict[0], tpr_dict[0], label='ROC Curve - ' + nombres[j] + ' - ' + roc_auc[j], color=colores[j], linestyle=':', linewidth=4)

            ax.plot([0, 1], [0, 1], 'k--', lw=2)
            ax.set_xlim([0.0, 1.0])
            ax.set_ylim([0.0, 1.05])
            ax.set_xlabel('False Positive Rate', fontsize=text_fontsize)
            ax.set_ylabel('True Positive Rate', fontsize=text_fontsize)
            ax.tick_params(labelsize=text_fontsize)
            ax.legend(loc='lower right', fontsize=text_fontsize)
    return ax

In [83]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
plot_roc3(Y, conprobas,TEST2, colores = colores, nombres = clasificadores, title="Multi-classifiers - ROC Curves" )
plt.show()

NameError: name 'TEST2' is not defined